<a href="https://colab.research.google.com/github/nonoumasy/Neighborhoods/blob/master/Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langdetect

     |████████████████████████████████| 983kB 6.7MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=7e15013256c9ff6b9e44e090707fd141dd1da0de65628b26ec706221618fc766
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [0]:
import pandas as pd

#import geopandas as gpd
import datetime as dt
import numpy as np
from langdetect import detect

pd.set_option('max_colwidth', 400)


In [0]:
listings_df = pd.read_csv('http://data.insideairbnb.com/portugal/lisbon/lisbon/2019-06-26/data/listings.csv.gz', low_memory=False)

In [0]:
#keep only relevant variables
listings_df = listings_df[[
                           'neighbourhood_cleansed',
                           'neighborhood_overview',
                           'latitude',
                           'longitude',
]]


#reduce neighborhoods
lis_neigh = ['Santa Maria Maior',
                'Misericrdia',
                'Estrela',
                'Alcntara',
                'Belm',
                'Ajuda',
                'Campo de Ourique',
                'Santo Antnio',
                'Arroios',
                'So Vicente',
                'Penha de Frana',
                'Beato',
                'Marvila',
                'Areeiro',
                'Alvalade',
                'Avenidas Novas',
                'Campolide',
                'So Domingos de Benfica',
                'Benfica',
                'Carnide',
                'Lumiar',
                'Olivais',
                'Parque das Naes',
                'Santa Clara']

## filter neighborhoods
listings_df.rename({'neighbourhood_cleansed' : 'neighbourhood'}, axis=1, inplace=True)
listings_df = listings_df[listings_df.neighbourhood.isin(lis_neigh)]

# drop null values
listings_df.dropna(inplace=True)

# sort by neighborhood and reset index
listings_df = listings_df.sort_values('neighbourhood').reset_index(drop=True)

In [5]:
listings_df

,neighbourhood,neighborhood_overview,latitude,longitude
0,Ajuda,"Considered the historical district of the Discoveries, it was from Belém that Vasco da Gama and many other navigators set off on the trips that would take them to the famous discoveries. So, in Belém, they were honored through wonderful museums and monuments that report their entire journey and making those who visit them, part of this wonderful history. Part of this story is World Heritage, b...",38.70505,-9.20125
1,Ajuda,You are very close to the palacio dr Belém and botanical garden.,38.70539,-9.20042
2,Ajuda,"Apartment is strategically located for those who wants to get most of their holidays spending time both visiting city and resting on the beach. Further, apartment has pedestrian access to all the Belem sightseeing points: Ajuda Palace, two Botanical Gardens (Royal and Tropical), Jerónimos Monastery, Car Museum, Monument of the Discoveries, Belém Tower among others. Being located in the reside...",38.70356,-9.20166
3,Ajuda,"The apartment is steps away from a supermarket, cafes, and eateries in a calm neighborhood near Belém. Alcântara’s vibrant nightlife scene is a short walk away. Pay a visit to the Museu de Marinha to see maritime exhibits and centuries-old barges.",38.70275,-9.19241
4,Ajuda,"Calm family district, you are among nice people. And few steps away from major National patrimonial buildings such as Palácio da Ajuda, Jardim Botânico da Ajuda, Museu dos Coches, Mosteiro dos Jerónimos, but i think you will be delighted to know you are near ""Pasteis de Belém""...huummmm",38.70562,-9.19971
...,...,...,...,...
12332,So Vicente,"The house is situated in a quiet, safe and familiar neighborhood. Very close to the historical center, Graça, Castelo and Alfama The house is situated in a quiet, safe and familiar neighborhood. Very close to the historical center, Graça, Castelo and Alfama",38.72248,-9.12564
12333,So Vicente,"Get ready to get lost in the picturesque streets of Alfama and meet its deepest and most well kept secrets. This genuine area is well connected and has a lot to offer on your doorstep. Coffee shops, Fado, Feira da Ladra flea market (Tuesdays and Saturdays), 28 tram and a great selection of restaurants are a short walk away. The neighborhood has a distinct village atmosphere; you can quickly fe...",38.71402,-9.12965
12334,So Vicente,"The apartment is located in the most historical Lisbon neighbourhood. With Lisbon views, the property is 650 m from the Santa Apolónia Train and Metro Stations. The surrounding Alfama area features various restaurants, eateries and Fado Houses, where guests may enjoy a Fado concert and Portuguese tapas. The Santa Justa Lift has scenic views over the city and is 1.4 km away. The iconic São Jorg...",38.71404,-9.12960
12335,So Vicente,"The apartment is located in a typical Portuguese neighborhood, in one of the 7 hills of Lisbon. Neighbor to several restaurants, cafes, grocery stores, supermarkets, shops. Public transport at the door. Next to the Castle, View points of Graça and Nossa Sra do Monte, Monastery of S. Vicente de Fora, National Pantheon, Alfama.",38.71813,-9.13140


In [0]:
listings_df.to_csv('lis.csv')

In [0]:
listings_df['lang'] = ''

#apply detect_lang to detect language or comment
for i in listings_df.neighborhood_overview:
  try:
    listings_df['lang'] = listings_df.neighborhood_overview.apply(detect)
    listings_df['lang'] = listings_df.lang.str.split(':', expand=True)[0]
  except:
    pass


In [0]:
ajuda_df = listings_df[listings_df.neighbourhood == 'Ajuda']

ajuda_df_eng = ajuda_df[ajuda_df.lang == 'en']

# drop duplicated text
ajuda_df_eng.neighborhood_overview.drop_duplicates(keep='first', inplace=True)

In [0]:
ajuda_df = listings_df[listings_df.neighbourhood == 'Ajuda']

ajuda_df['lang'] = ''

#apply detect_lang to detect language or comment
for i in ajuda_df.neighborhood_overview:
  try:
    ajuda_df['lang'] = ajuda_df.neighborhood_overview.apply(detect)
    ajuda_df['lang'] = ajuda_df.lang.str.split(':', expand=True)[0]
  except:
    pass


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [94]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')

first_vector_tfidfvectorizer = vectorizer.fit_transform(ajuda_df_eng.neighborhood_overview)[0]


df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=vectorizer.get_feature_names())
df.sort_values(0, ascending=False)[0:50]
 

,0
wonderful,0.256169
history,0.239014
famous,0.219148
belém,0.212313
world,0.184373
discoveries,0.184373
thinks,0.140174
making,0.140174
born,0.140174
mainly,0.140174


In [79]:
for index, text in enumerate(ajuda_df_eng.neighborhood_overview):
  print(index, text);


0 Considered the historical district of the Discoveries, it was from Belém that Vasco da Gama and many other navigators set off on the trips that would take them to the famous discoveries. So, in Belém, they were honored through wonderful museums and monuments that report their entire journey and making those who visit them, part of this wonderful history. Part of this story is World Heritage, but fool anyone who thinks Belém is just history. To sweeten your mouth, it is imperative that you visit the famous Confeitaria de Belém (mainly known as "Pastéis de Belém"). The famous cream pastries were born here and continue to be created, through a secret recipe, conferring them the status of "the best in the world". Nearby, up the hill, there is the Ajuda neighborhood. In addition to the beautiful botanical garden, you will also find the unique Ajuda National Palace.
1 You are very close to the palacio dr Belém and botanical garden.
2 Apartment is strategically located for those who wants t